In [1]:
import glob
import pandas as pd
# import numpy as np

In [9]:
relations = pd.read_csv("data\World2022-04-17.csv", header=None)
relations = relations.rename(columns={0:"Viewers_list", 1:"Streamer", 2:"Date", 3:"Category", 4:"Viewers", 5:"Language"})

In [10]:
relations["Viewers"].min()

2032

In [39]:
# 1m40 
# all_relations = glob.glob("mars13-20/relations*.csv")
# relations = pd.concat((pd.read_csv(f, header=None) for f in all_relations), axis=0, ignore_index=True)
# relations = relations.rename(columns={0:"Viewers_list", 1:"Streamer", 2:"WatchTime"})
# print(f" ** Memory usage of the file - {sum(relations.memory_usage()) * 0.000001} MB for {len(relations.index)} Rows")

In [7]:
# 2m20s rend la liste "list()"
relations['Viewers_list'] = relations['Viewers_list'].apply(eval)

In [8]:
# relations = relations[relations['Viewers_list'].apply(lambda x: len(x) >= 10000)]

In [10]:
relations.head()

,Viewers_list,Streamer,Date,Category,Viewers,Language
0,"[0000moonlight, 001133557799, 0025, 003ny, 007...",Emiru,2022-04-01 00:00:03,Just Chatting,45368,en
1,"[000_fnwortas_000, 008alexr, 00animatech, 00bu...",ROSHTEIN,2022-04-01 00:00:03,Slots,34077,en
2,"[000jp, 00od, 00saintjimmy00, 02hiiro, 02lynne...",HasanAbi,2022-04-01 00:00:03,Just Chatting,30861,en
3,"[000008, 003ray_, 00601289, 007225, 007davis_t...",TSM_ImperialHal,2022-04-01 00:00:03,Apex Legends,29138,en
4,"[0000_kevin0000, 000maurojunior000, 006627, 00...",alanzoka,2022-04-01 00:00:03,Midnight Ghost Hunt,28314,pt


In [11]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [12]:
# 45s 
viewers_1D = to_1D(relations["Viewers_list"])
viewers_1D_count = viewers_1D.value_counts()

In [63]:
# WARNING DO NOT EXECUTE TWICE
# viewers_1D.to_csv("mars13-20/viewers1D.csv", mode='a')

In [14]:
viewers_1D_count

commanderroot     9260
spiketrapclair    9039
kaxips06          8302
aten              7977
elysian           7809
                  ... 
tgfujfdry            1
tgrva11              1
th_gitt              1
the_cosmic_egg       1
zharfantea17         1
Length: 7527091, dtype: int64

Les valeurs à suivre sont en quarts d'heure.

In [64]:
viewers_1D_count.head(3) # les bots :)

commanderroot     1818
spiketrapclair    1682
aten              1594
dtype: int64

In [15]:
viewers_1D_count.describe()

count    7.527091e+06
mean     9.257856e+00
std      1.712691e+01
min      1.000000e+00
25%      2.000000e+00
50%      5.000000e+00
75%      1.100000e+01
max      9.260000e+03
dtype: float64

In [66]:
# Quarts_heure = (np.datetime64("2022-02-14 00:00")-np.datetime64("2022-02-13 00:00")).astype(int)/15
# Quarts_heure
Quarts_heure = 96

In [67]:
def temps(x, streamersRegardés=2):
    print(f"{x/4:.2f} heures regardées, soit {x/4/24:.2f} jours sur un total de {Quarts_heure/4/24:.2f} jours étudiés.")
    print(f"Ou encore {streamersRegardés} streamers regardés en parallèle pendant {(x/streamersRegardés)/(Quarts_heure/24):.2f} heures par jour.")

In [68]:
temps(viewers_1D_count.quantile(0.9999), 4)

45.75 heures regardées, soit 1.91 jours sur un total de 1.00 jours étudiés.
Ou encore 4 streamers regardés en parallèle pendant 11.44 heures par jour.


1 personne sur 10000 est soit un bot, soit un dégénéré qui mutliplexe 2 streams en même temps pendant 19h par jour.

In [69]:
def RejectedPseudo(minHeure=1):
    temp = list(viewers_1D_count[viewers_1D_count>viewers_1D_count.quantile(0.9999)].index) + list(viewers_1D_count[viewers_1D_count<4*minHeure].index)
    print(f"{len(temp)} viewers rejetés sur {len(viewers_1D_count)}")
    return temp

In [70]:
exiled = RejectedPseudo()

2255254 viewers rejetés sur 4855261


In [71]:
# 35s c'est insane pandas ptdr
gephi = relations[['Viewers_list', 'Streamer']].explode('Viewers_list')
gephi = gephi.rename(columns={'Viewers_list': 'source', 'Streamer': 'target'})

In [72]:
# 30s
gephi = gephi[~gephi['source'].isin(exiled)]

In [ ]:

streamerviewer = gephi['target'].unique()
gephi = gephi[~gephi['source'].isin(streamerviewer)]

In [73]:
gephi['temp']=1

In [74]:
# 1m20s
gephi = gephi.groupby(by=['source', 'target']).agg({'temp':'count'})
gephi = gephi.reset_index().rename(columns={'temp': 'weight'})

In [75]:
gephi.head()

,source,target,weight
0,000000000000,Distortion2,1
1,000000000000,Jerma985,7
2,000000000000,forsen,1
3,00000000000000,xQcOW,8
4,0000000000000000,Ninja,4


In [76]:
gephi.shape

(4075553, 3)

In [25]:
# 30s
gephi.to_csv('twitchGephi.csv', mode='w', encoding='utf-8')